在日常的事务处理中，我们常常会遇到这样的情况：某个活动或者报名需要选择班级中的几名同学，当同学们提交了自己的姓名或学号之后，我们要对所需的信息进一步结合手头上有的数据库进行补全。

![title](img/p1.png)

<center>将下方的数据补充到上方的报名表中</center>

![title](img/p2.png)

# 第三方库pywin32 

可以结合API对我们的常用软件excel、word等软件进行处理，实现基本的读取写入，通过合理的代码实现，我们就能将平时繁复的人工填表操作交给代码

我们可以直接使用pip命令对第三方库进行安装

pip install -i pypiwin32

如果觉得下载速度较慢，可以使用国内镜像源进行下载，此方法适用于大多数python库

pip install -i https://pypi.doubanio.com/simple/ pypiwin32



# 1.导入第三方库、构建类和函数

成功运行下方的代码导入第三方库后，只需要构建相应的类，就可以使用其中的函数。

In [15]:
import os
import sys
import string
import operator
import win32com.client

class easyExcel:
    def __init__(self, filename=None):
        self.xlApp = win32com.client.Dispatch('Excel.Application')
        if filename:
            self.filename = filename
            self.xlBook = self.xlApp.Workbooks.Open(filename)
        else:
            self.xlBook = self.xlApp.Workbooks.Add()
            self.filename = '' 
   
    def save(self, newfilename=None):
        #保存
        if newfilename:
            self.filename = newfilename
            self.xlBook.SaveAs(newfilename)
        else:
            self.xlBook.Save() 
    def close(self):
        #关闭
        self.xlBook.Close(SaveChanges=0)
        del self.xlApp
       
    def getCell(self, sheet, row, col):
        #读取
        "Get value of one cell"
        sht = self.xlBook.Worksheets(sheet)
        return sht.Cells(row, col).Value
    def setCell(self, sheet, row, col, value):
        #写入
        "set value of one cell"
        sht = self.xlBook.Worksheets(sheet)
        sht.Cells(row, col).Value = value
    def getRange(self, sheet, row1, col1, row2, col2):
        #范围读取
        "return a 2d array (i.e. tuple of tuples)"
        sht = self.xlBook.Worksheets(sheet)
        return sht.Range(sht.Cells(row1, col1), sht.Cells(row2, col2)).Value
   
    def setRange(self, sheet, leftCol, topRow, data):
        #范围插入
        """insert a 2d array starting at given location.
        Works out the size needed for itself""" 
        bottomRow = topRow + len(data) - 1
        rightCol = leftCol + len(data[0]) - 1
        sht = self.xlBook.Worksheets(sheet)
        #清空所当前sheet里的有单元格
        sht.Cells.Clear()
        sht.Range(
            sht.Cells(topRow, leftCol),
            sht.Cells(bottomRow, rightCol)
            ).Value = data
        sht.Range(
            sht.Cells(topRow, leftCol),
            sht.Cells(topRow, rightCol)
            ).Font.Bold = True
       
    def getContiguousRange(self, sheet, row, col):
        """Tracks down and across from top left cell until it
        encounters blank cells; returns the non-blank range.
        Looks at first row and column; blanks at bottom or right
        are OK and return None witin the array"""
        sht = self.xlBook.Worksheets(sheet)
        # find the bottom row
        bottom = row
        while sht.Cells(bottom + 1, col).Value not in [None, '']:
            bottom = bottom + 1
        # right column
        right = col
        while sht.Cells(row, right + 1).Value not in [None, '']:
            right = right + 1
        #设置第一行若干列为粗体
        return sht.Range(sht.Cells(row, col), sht.Cells(bottom, right)).Value

# 2.创建对象，使用函数

下方代码的含义是：我们将filename路径的excel文件，将该文件创建成一个特定的对象。

如何理解类和对象？比如我们有一个名为小狗（easyExcel）的类，那我们就可以创建一个名为哈士奇（excel001）的独一无二的对象，并对其使用相应的函数。

excel001 = easyExcel(filename)#创建对象
excel001.getCell("Sheet1",i,2)#获取Sheet1中第i行第2列的数据
excel001.setCell("Sheet1",x,6,value)#向Sheet1中第x行第6列写入数据value

**下面我们演示如何将数据从通讯录中填入到报名表中**

In [1]:
filename1 = r"E:\应用\python\jupyter\报名表.xlsx"
filename2 = r"E:\应用\python\jupyter\通讯录.xls"

excel1 = easyExcel(filename1)
excel2 = easyExcel(filename2)
for i in range (2,10):
    temp1=excel1.getCell("Sheet1",i,1)#读取需要收集信息的学生序号
    for x in range (2,42):
        temp2=excel2.getCell("Sheet1",x,1)#寻找通讯录中的相应位置
        if temp1==temp2:
            print("匹配成功")
            excel1.setCell("Sheet1",i,2,excel2.getCell("Sheet1",x,2))#将值设置为excel1.getCell("Sheet1",x,2)
            excel1.setCell("Sheet1",i,3,excel2.getCell("Sheet1",x,4))
            break

excel2.save(filename1)
excel1.close()#将已经打开的进程关闭



NameError: name 'easyExcel' is not defined

![title](img/p3.png)

In [2]:
#保护隐私,将通讯录中的号码去掉一位
filename1 = r"E:\应用\python\jupyter\通讯录.xls"
filename2 = r"E:\应用\python\jupyter\新通讯录.xls"
excel1 = easyExcel(filename1)

for i in range (2,42):
    excel1.setCell("Sheet1",i,4,str(excel1.getCell("Sheet1",i,4))[:-3])
    print("替换成功")

excel1.save(filename2)
excel1.close()

NameError: name 'easyExcel' is not defined

# 3.word操作

使用方法与上面相同，也是通过创建word对象进行操作，具体函数及代码如下。

In [ ]:
import os
import win32com.client
import time

class RemoteWord:
    def __init__(self, filename=None):
        self.xlApp = win32com.client.DispatchEx('Word.Application')
        self.xlApp.Visible = 0
        self.xlApp.DisplayAlerts = 0  # 后台运行，不显示，不警告
        if filename:
            self.filename = filename
            if os.path.exists(self.filename):
                self.doc = self.xlApp.Documents.Open(filename)
            else:
                self.doc = self.xlApp.Documents.Add()  # 创建新的文档
                self.doc.SaveAs(filename)
        else:
            self.doc = self.xlApp.Documents.Add()
            self.filename = ''

    def add_doc_end(self, string):
        '''在文档末尾添加内容'''
        rangee = self.doc.Range()
        rangee.InsertAfter('\n' + string)

    def add_doc_start(self, string):
        '''在文档开头添加内容'''
        rangee = self.doc.Range(0, 0)
        rangee.InsertBefore(string + '\n')

    def insert_doc(self, insertPos, string):
        '''在文档insertPos位置添加内容'''
        rangee = self.doc.Range(0, insertPos)
        if (insertPos == 0):
            rangee.InsertAfter(string)
        else:
            rangee.InsertAfter('\n' + string)

    def replace_doc(self, string, new_string):
        '''替换文字'''
        self.xlApp.Selection.Find.ClearFormatting()
        self.xlApp.Selection.Find.Replacement.ClearFormatting()
        self.xlApp.Selection.Find.Execute(string, False, False, False, False, False, True, 1, True, new_string, 2)

    def save(self):
        '''保存文档'''
        self.doc.Save()

    def save_as(self, filename):
        '''文档另存为'''
        self.doc.SaveAs(filename)

    def close(self):
        '''保存文件、关闭文件'''
        self.save()
        self.xlApp.Documents.Close()
        self.xlApp.Quit()

# 4.对文件夹内的所有word文档进行操作，将其中的小明替换为小张

In [ ]:
#重复运行需要关闭word进程
if __name__ == '__main__':
    print("批量替换过程中可能会一分钟都没有成功替换，请耐心等待")
    path = "C:/Users/13736/Desktop/实验报告重命名版/" # 文件夹路径 必须是反斜杠/
    files = []
    for file in os.listdir(path):
        if file.endswith(".docx"):  # 排除文件夹内的其它干扰文件，只获取word文件
            files.append(path + file)
    for i in files:
        docx=i
        doc = RemoteWord(docx)  # 初始化一个doc对象
         #替换内容
        doc.replace_doc('小明', '小张')  # 替换文本内容
        doc.save()
        doc.close()
        print(i,"完成替换")